In [1]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QComboBox, QPushButton, QVBoxLayout, QWidget, QFileDialog, QMessageBox
from PyQt5.QtGui import QPixmap, QImage
import cv2
import numpy as np
import os
from PIL import Image
import time
from split_bregman import split_bregman
from ADMM import ADMM
from chambolle_pd import tv_denoise_chambolle
from FGP import FGP
from ADMM_color import ADMM_3D
from ROF_proposed import ROFtv


# The main application class
class ImageDenoisingApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Image Denoising App")
        self.image_path = None
        self.algorithm = None
        self.process_flag = None
        self.N = 500 # maxiter
        self.weight = 12.5 # 1/lambda
        self.iter_num = 0 # number of iteration to reach convergence
        self.image_shape = (0,0)
        # Create the central widget and layout
        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)
        self.layout = QVBoxLayout(self.central_widget)

        # Add a label to display the image
        self.image_label = QLabel(self)
        self.layout.addWidget(self.image_label)

        # Add a combo box to select the image type
        self.image_type_combo = QComboBox(self)
        self.image_type_combo.addItem("B&W")
        self.image_type_combo.addItem("Colour")
        self.image_type_combo.currentIndexChanged.connect(self.image_type_changed)
        self.layout.addWidget(self.image_type_combo)

        # Add a combo box to select the algorithm
        self.algorithm_combo = QComboBox(self)
        self.layout.addWidget(self.algorithm_combo)

        # Add a button to upload the image
        self.upload_button = QPushButton("Upload Image", self)
        self.upload_button.clicked.connect(self.upload_image)
        self.layout.addWidget(self.upload_button)
        

        # Add a button to process the image
        self.process_button = QPushButton("Process Image", self)
        self.process_button.clicked.connect(self.process_image)
        # self.process_button.clicked.connect(self.plot_image)
        self.layout.addWidget(self.process_button)

        # Add a button to show image denoising dynamics
        self.dynamics_button = QPushButton("Processing Dynamics", self)
        # self.process_button.clicked.connect(self.process_image)
        self.dynamics_button.clicked.connect(self.plot_image)
        self.layout.addWidget(self.dynamics_button)

        # Add a button to download the image
        self.download_button = QPushButton("Download Image", self)
        self.download_button.clicked.connect(self.download_image)
        self.layout.addWidget(self.download_button)

        # Update the algorithm options based on the selected image type
        self.update_algorithm_options("B&W")

    # Update the algorithm options when the image type changes
    def image_type_changed(self, index):
        image_type = self.image_type_combo.itemText(index)
        self.update_algorithm_options(image_type)

    # Update the algorithm options based on the selected image type
    def update_algorithm_options(self, image_type):
        self.algorithm_combo.clear()
        if image_type == "B&W":
            self.algorithm_combo.addItem("split_bregman")
            self.algorithm_combo.addItem("ADMM")
            self.algorithm_combo.addItem("PD")
            self.algorithm_combo.addItem("FGP")
            #self.algorithm_combo.addItem("ROF")
        else:
            self.algorithm_combo.addItem("FGP")
            #self.algorithm_combo.addItem("PD")
            self.algorithm_combo.addItem("ADMM")

    # Open a file dialog to upload an image
    def upload_image(self):
        file_dialog = QFileDialog(self)
        self.image_path, _ = file_dialog.getOpenFileName(self, "Upload Image", "", "Image Files (*.png *.jpg *.jpeg)")
        if self.image_path:
            # s = denoised_image.shape
            pixmap = QPixmap(self.image_path)
            s = pixmap.size()
            alpha = s.width()/s.height()
            height_const = 512
            self.image_label.setPixmap(pixmap.scaled(int(alpha*height_const), int(height_const)))

    # Process the uploaded image using the selected algorithm
    def process_image(self):
        self.process_flag = None
        if not self.image_path:
            QMessageBox.warning(self, "Warning", "Please upload an image first.")
            return None
        if self.image_type_combo.currentText() == "B&W":
            image = cv2.imread(self.image_path, cv2.IMREAD_GRAYSCALE)
            s = image.shape
            self.image_shape = [s[0], s[1]]
            # if s[-1] >= 2 :
            #     QMessageBox.warning(self, "Warning", "Please use color image option.")
            #     image = cv2.imread(self.image_path, cv2.IMREAD_GRAYSCALE)
            # else:
            #     image = image.reshape(s[0], s[1])
            
            self.algorithm = self.algorithm_combo.currentText()
            if self.algorithm == "split_bregman":
                # resize image
                resize_w = max(self.image_shape[0], self.image_shape[1])
                image = cv2.resize(image, (resize_w, resize_w), interpolation = cv2.INTER_AREA)
                denoised_image, self.iter_num = split_bregman(image, self.weight, self.N, s, isotropic=False)
                denoised_image = cv2.resize(denoised_image, (512, int(512*(s[0]/s[1]))), interpolation = cv2.INTER_AREA)
            elif self.algorithm == "ADMM":
                denoised_image, self.iter_num = ADMM(image, self.weight, self.N, isotropic=False)
                self.process_flag = True # processing finished
            elif self.algorithm == "PD":
                denoised_image, self.iter_num = tv_denoise_chambolle(image, self.weight, N=self.N)
                self.process_flag = True
            elif self.algorithm == "FGP":
                denoised_image, self.iter_num = FGP(image, self.weight, self.N, isotropic=False)
            else:
                QMessageBox.warning(self, "Warning", "Invalid algorithm selected.")
                return None
        else:
            self.algorithm = self.algorithm_combo.currentText()
            image = cv2.imread(self.image_path)
            s = image.shape
            self.image_shape = [s[0], s[1]]
            if self.algorithm == "FGP":
                denoised_image, self.iter_num = FGP(image, self.weight, self.N, channel_axis = True)
                # print(attr)
            elif self.algorithm == "ADMM":
                denoised_image, self.iter_num = ADMM_3D(image, self.weight, self.N, channel_axis = True)
            else:
                QMessageBox.warning(self, "Warning", "Invalid algorithm selected.")
                return None
        # return denoised_image, denoised_image_set
    # def display_image(denoised_image):
        # Display the denoised image with new window
        # print the denoised image for 15 seconds
        #TODO：迭代停止出现弹窗
        msgBox = QMessageBox()
        msgBox.setText("Iteration has ended, please view obtained image.");
        msgBox.exec_();
        denoised_image = denoised_image.astype(np.uint8)
        s = denoised_image.shape
        alpha = s[0]/s[1]
        height_const = 512
        #self.image_label.setPixmap(pixmap.scaled(alpha*height_const, height_const))
        cv2.imwrite("denoised_image.jpg", denoised_image)
        pixmap = QPixmap("denoised_image.jpg")
        self.image_label.setPixmap(pixmap.scaled(int(height_const/alpha), int(height_const)))
    def plot_image(self):
        # access iteration time
        assert self.iter_num > 0, "error: no denoising process available."

        # display image for 30 seconds sum(waitkey) = 30000
        num_waitkey = 10000//self.iter_num
        for i in range(self.iter_num+1):
            # Create the path to the temporary image file
            path_temp = "_".join([str(i),"temp.npy"])
            # Check if the temporary image file exists
            if os.path.exists(path_temp):
                print("displaying image", i)
                # Load the image
                deno_temp = np.load(path_temp)
                # display image
                # Convert the image to the appropriate color format
                if self.image_type_combo.currentText() == "B&W":
                    frame = cv2.cvtColor(deno_temp, cv2.COLOR_GRAY2RGB)
                else:
                    frame = deno_temp
                    # frame = cv2.cvtColor(deno_temp, cv2.COLOR_RGB2RGB)
                # Get the height and width of the image
                h, w = self.image_shape[0], self.image_shape[1]
                
                alpha = w/h
                h = 512
                w = int(h*alpha)
                # Calculate the bytes per line
                bytesPerLine = 3*w

                # Create a QImage object from the image data
                qimg = QImage(frame.data,w,h,bytesPerLine, QImage.Format.Format_RGB888)
                # Display the frame
                cv2.imshow("Temporary Image", frame)
                # Pause for 0.5 seconds
                cv2.waitKey(num_waitkey)
                # Remove the temporary image file
                os.remove(path_temp)
        cv2.destroyAllWindows()
        # Save the denoised 
    #def save_image(denoised_image):


    # Open a file dialog to download the denoised image
    def download_image(self):
        #TODO: delete remporary npy files if still existent

        if not self.image_path:
            QMessageBox.warning(self, "Warning", "No denoised image available.")
            return
        file_dialog = QFileDialog(self)
        file_dialog.setAcceptMode(QFileDialog.AcceptSave)
        file_path, _ = file_dialog.getSaveFileName(self, "Save Denoised Image", "", "Image Files (*.png *.jpg *.jpeg)")
        
        if file_path:
            # delete temp files
            for i in range(self.iter_num+1):
            # Create the path to the temporary image file
                path_temp = "_".join([str(i),"temp.npy"])
                # Check if the temporary image file exists
                if os.path.exists(path_temp):
                
                    # Remove the temporary image file
                    os.remove(path_temp)
            cv2.imwrite(file_path, cv2.imread("denoised_image.jpg"))
            QMessageBox.information(self, "Success", "Denoised image saved successfully.")

    # Show the application window
    def show(self):
        super().show()
        self.resize(600, 600)

 # Run the application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    image_denoising_app = ImageDenoisingApp()
    image_denoising_app.show()
    sys.exit(app.exec_())

(658, 1170, 3, 2)
dtz= [[[ 0.42352941 -0.37647059 -0.47058824]
  [ 0.70980392 -0.28627451 -0.09019608]
  [ 1.62745098  0.71764706  1.16470588]
  ...
  [-1.25490196  0.76470588  0.50196078]
  [-0.59607843  0.37254902  0.02352941]
  [-0.06666667  0.1254902  -0.14901961]]

 [[-0.50196078  0.52941176  0.4745098 ]
  [-0.55294118  0.33333333  0.12941176]
  [-0.50588235  0.10196078 -0.83921569]
  ...
  [ 1.08235294 -1.         -0.8       ]
  [ 0.36862745 -0.81960784 -0.01176471]
  [-0.15294118 -0.41960784 -0.01176471]]

 [[ 0.11372549 -0.34509804  0.16470588]
  [ 0.10588235  0.2745098   0.23137255]
  [-0.11372549 -0.21960784  0.21960784]
  ...
  [-1.47843137 -0.14509804  0.23137255]
  [ 0.14117647  0.74901961  0.19607843]
  [ 0.43529412  0.51764706  0.1372549 ]]

 ...

 [[ 0.26666667  0.56862745 -0.30196078]
  [-0.02745098  0.46666667 -0.45490196]
  [ 0.1372549   0.89019608 -1.35294118]
  ...
  [ 0.49803922 -0.99607843 -0.04313725]
  [-0.43529412 -0.83529412 -0.4627451 ]
  [-0.76470588 -0.498

/Users/liangxinyu/Downloads/TV-L2-Image-Denoising-2/ADMM_color.py:88: ComplexWarning: Casting complex values to real discards the imaginary part
  x_next[:,:,0] = x_solver(z[:,:,0,:], u[:,:,0,:], u0_R, rho, deno)
/Users/liangxinyu/Downloads/TV-L2-Image-Denoising-2/ADMM_color.py:90: ComplexWarning: Casting complex values to real discards the imaginary part
  x_next[:,:,1] = x_solver(z[:,:,1,:], u[:,:,1,:], u0_G, rho, deno)
/Users/liangxinyu/Downloads/TV-L2-Image-Denoising-2/ADMM_color.py:92: ComplexWarning: Casting complex values to real discards the imaginary part
  x_next[:,:,2] = x_solver(z[:,:,2,:], u[:,:,2,:], u0_B, rho, deno)


ite 0


AssertionError: error: no denoising process available.

AssertionError: error: no denoising process available.

SystemExit: 0

/Users/liangxinyu/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3386: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
